# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3) - Parte 1: Preparación de los datos

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn.cluster import KMeans, MeanShift
from sklearn import manifold, preprocessing, decomposition

# Funciones propias
from utils_limpieza import * 

# Clear preferencias
plt.rcdefaults()
pd.reset_option('^display\.float_format')

---
# Preparación de Datos <span style="color:magenta">**(Paso 1)**</span>

Se repiten los pasos de curación realizados en la entrega anterior. A tener en cuenta:
* Algunos pasos que se hacían por separado, ahora se hacen todos juntos. 
* No se simplifica la variable `Deposito`, ya que se descarta desde el principio.
* lo del 10K
* No se hace el cálculo de promedio y varianza de los vectores finales.

1. Cargamos el dataset crudo.

In [ ]:
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas_raw = pd.read_csv(path)

2. Eliminamos las siguientes variables:
    * `INSCRIPCION`.
    * `CATEGORIA`.
    * `DESCRIPCION_CATEGORIA`.
    * `CATEGORIA (Ajustado)`.
    * `NOMBRE`.
    * `'CM04`.
    * `DESC_TRATAMIENTO_FISCAL`.
    * `TRATAMIENTO_DIFERNCIAL`.
    * `TRATAMIENTO_FISCAL`.
    * `PORCENTAJE_COMISION_EMPRESA`.
    * `DEPOSITO`.

In [ ]:
ventas = limpiar_basic(ventas_raw, cols_drop=['INSCRIPCION', 'CATEGORIA', 
                                                  'DESCRIPCION_CATEGORIA', 
                                                  'CATEGORIA (Ajustado)', 
                                                  'NOMBRE', 'CM04', 'DEPOSITO',
                                                  'DESC_TRATAMIENTO_FISCAL', 
                                                  'TRATAMIENTO_DIFERNCIAL', 
                                                  'TRATAMIENTO_FISCAL', 
                                                  'PORCENTAJE_COMISION_EMPRESA'])

3. Renombramos como `Otros` las subcategorías que no tengan al menos 1 `MODELO` = 1.

In [ ]:
ventas = renombrar_elementos(ventas, columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

4. Eliminamos los registros que contienen las siguientes subcategorías:
    * `Otros`.
    * `Instalación, Mantenimiento, Reparación, etc de productos varios` (a.k.a. `Mantenimiento`).

In [ ]:
ventas = ventas[ventas['SUB-CATEGORIA'] != 'Otros'].copy()
ventas = ventas[ventas['SUB-CATEGORIA'] != 'Instalación, Mantenimiento, Reparación, etc de productos varios'].copy()

5. Creamos la variable `Fecha`, que surge como:
    $$Fecha = Año + Mes$$

    Luego, eliminamos las variables `MES` y `AÑO`.

In [ ]:
ventas["Fecha"] = pd.to_datetime(ventas['MES'].astype(str) + '-' + ventas['AÑO'].astype(str), format='%m-%Y')
ventas = limpiar_basic(ventas, cols_drop=['MES', 'AÑO'])

6. Anonimizamos la variable sensible `ID_VENDEDOR`.

In [ ]:
ventas, _ = anonimizar(ventas, 'ID_VENDEDOR')

7. Simplificamos el nombre de las variables.

In [ ]:
with open("../references/tp2_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas.rename(columns = column_dict, inplace = True)

8. Simplificamos las categorías en `Subrubro`.

In [ ]:
with open("../references/tp2_subrubro_dict.json") as subrubro_dict_json:
    subrubro_dict = json.load(subrubro_dict_json)

ventas['Subrubro'] = ventas['Subrubro'].replace(subrubro_dict)

9. <span style="color:magenta">**(Este paso es nuevo)**</span> Remplazamos con $0 a todas las ventas y comisiones que estén en el rango desde -$10.000 hasta $10.000.

In [ ]:
ventas[['Ventas', 'Comision']].describe()

In [ ]:
cut = 10000
a = ventas[(ventas['Ventas'] < cut) & (ventas['Ventas'] > - cut)].copy()
b = a[a['Ventas']==0].copy()
print(f'Se trata de {a.shape[0]} registros. De los cuales {b.shape[0]} ya son idénticamente nulos.')
print(f'Es decir que tenemos {a.shape[0] - b.shape[0]} registros que no son nulos dentro de este rango (${-cut}, ${cut}).')
c = a[a['Ventas'] != 0].copy()
cpos = c[c['Ventas']>0]
cneg = c[c['Ventas']<0]
print(f'De los valores no nulos, hay {cpos.shape[0]} positivos y {cneg.shape[0]} negativos.')

sns.histplot(cpos['Ventas'])
sns.histplot(cneg['Ventas'])
plt.show()

In [ ]:
# Imputamos las ventas
ven = ventas['Ventas'].copy()
ven.mask((ven < cut) & (ven > - cut), inplace=True)
ven.fillna(0, inplace=True)
ventas['Ventas'] = ven

# Imputamos las comisiones
ven = ventas['Comision'].copy()
ven.mask((ven < cut) & (ven > - cut), inplace=True)
ven.fillna(0, inplace=True)
ventas['Comision'] = ven

10. Eliminamos el efecto de la inflación. El procedimiento es el siguiente:
    $$\text{VAR}_{mm-aaaa} \frac{\text{IPC}_{06-2022}}{\text{IPC}_{mm-aaaa}}$$

    Para todo valor de cada variable (VAR) correspondiente a cierto mes "mm-aaaa" se lo divide por el Índice de Precios al Consumidor (IPC) correspondiente a ese mes "mm-aaaa" y luego se lo multiplica por el IPC correspondiente al mes "06-2022". De esta forma, todos los valores de cada variable VAR van a quedar expresados en unidades monetarias del "06-2022".

In [ ]:
precios_path = "../data/external/tp2_IPC_Indec.csv"

if exists(precios_path):
    print('Este archivo ya existe.')
else:
    print('Este archivo no existe: ¡Vamos a crearlo!')
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_06_23.xls"
    df = pd.read_excel(url, sheet_name="Índices IPC Cobertura Nacional", header=None, usecols="B:CA", skiprows=[0, 1, 2, 3, 4, 6, 7, 8], nrows=2)
    df = df.transpose()
    df.columns = ["Fecha", "INDICE"]
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df.to_csv(precios_path, index=False)

precios = pd.read_csv(precios_path)

# Hay que asegurar que la variable clave tenga el mismo tipo en los dos dataframes
precios["Fecha"] = pd.to_datetime(precios["Fecha"])

ventas = ventas.merge(precios[["Fecha", "INDICE"]], on="Fecha", how="left")

In [ ]:
# Hacemos la imputación
indexar(ventas, 'Ventas')
indexar(ventas, 'Comision')

In [ ]:
# Se descartan las variables que no se usan
ventas = limpiar_basic(ventas, cols_drop=['Ventas', 'Comision', 'INDICE'])

# Se renombran las variables
ventas.rename(columns = {'Ventas_Real': 'Ventas', 
                              'Comision_Real': 'Comision'}, inplace = True)

# Se reacomodan las columnas
ventas = ventas[['ID', 'Omega', 'Subrubro', 'Fecha', 'Ventas', 'Comision', 'Modelo']]

ventas = ventas.sort_values(['Fecha', 'Ventas']).reset_index(drop=True)

11. Para cada combinación posible de `Subrubro`, `ID` y `Fecha`, vamos a:
    * Sumar todos los valores de `Ventas`.
    * Sumar todos los valores de `Comision`.
    * Mantener los valores de `Modelo` y de `Omega`.

    Al hacer esto, se agregan en una única fila todas las observaciones que pertenezcan a un mismo vendedor en una dada fecha bajo un cierto subrubro.

In [ ]:
agregado = ventas.groupby(['Subrubro', 'ID', 'Fecha']).agg({
    'Omega': 'min',
    'Ventas': 'sum',
    'Comision': 'sum',
    'Modelo': 'min'
}).reset_index()

12. Se eliminan los registros asociados a vendedores que tienen siempre ventas nulas dentro de un mismo subrubro.

In [ ]:
agregado_no_nulo = agregado.groupby(['Subrubro', 'ID']).filter(lambda x: (x['Ventas'] != 0).any()).copy()

13. Deberíamos tener 42 observaciones por par ID/Subrubro. Imputamos los meses faltantes con valor 0 en ventas y en comisión.

In [ ]:
# Distinguimos los valores originales de los imputados (`Dato_original`).
agregado_no_nulo['Dato_original'] = 1

# Contar las observaciones por combinación de ID y Subrubro
conteo_combinaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().reset_index(name='Conteo')

# Filtrar las combinaciones con menos de 42 observaciones
combinaciones_faltantes = conteo_combinaciones[conteo_combinaciones['Conteo'] < 42]

# Lista para almacenar las observaciones faltantes
observaciones_faltantes = []

# Iterar sobre las combinaciones faltantes
for _, combinacion in combinaciones_faltantes.iterrows():
    id_val = combinacion['ID']
    subrubro_val = combinacion['Subrubro']
    
    # Obtener fechas existentes y fechas faltantes
    fechas_existentes = agregado_no_nulo[(agregado_no_nulo['ID'] == id_val) & (agregado_no_nulo['Subrubro'] == subrubro_val)]['Fecha']
    fechas_faltantes = set(agregado_no_nulo['Fecha'].unique()) - set(fechas_existentes)
    
    # Agregar observaciones faltantes al DataFrame
    for fecha_faltante in fechas_faltantes:
        observacion = {
            'ID': id_val,
            'Subrubro': subrubro_val,
            'Fecha': fecha_faltante,
            'Ventas': 0,
            'Comision': 0,
        }
        observaciones_faltantes.append(observacion)

# Crear DataFrame con las observaciones faltantes
df_observaciones_faltantes = pd.DataFrame(observaciones_faltantes)

# Agregar observaciones faltantes al DataFrame agregado_no_nulo
agregado_limpio = pd.concat([agregado_no_nulo, df_observaciones_faltantes], ignore_index=True)

# Imputo los valores Omega y Modelo asociados a los nuevos meses adicionados. Es más eficiente agregarlo en una celda aparte y no en el for (antes demoraba 9 minutos, ahora menos de 1).
agregado_limpio['Omega'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Omega'].transform('max')
agregado_limpio['Modelo'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Modelo'].transform('max')

# Relleno los valores nulos de Dato_original con 0.
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].fillna(0)

# Convertir las columnas 'Omega', 'Modelo' y 'Dato_original' a tipo int64
agregado_limpio['Omega'] = agregado_limpio['Omega'].astype('int64')
agregado_limpio['Modelo'] = agregado_limpio['Modelo'].astype('int64')
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].astype('int64')

promedio_observaciones = agregado_limpio.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(f'Cade vendedor tiene asociados {promedio_observaciones} meses diferentes.')

14. Ordenar luego de todo lo que se hizo. Dejar que primero esté Fecha. Sirve para el pivoteo que hacemos después.

In [ ]:
registros_vendedores_abs = agregado_limpio.sort_values(['Fecha', 'Subrubro', 'ID']).reset_index(drop=True)

15. Queremos hacer el cálculo de la varación porcentual. Con el dataset como está actualmente (presencia de ceros), pueden generarse resultados que tienden a infinito. Para superar esta situación, se reemplazan todos los ceros por NaN.

In [ ]:
registros_vendedores_rel = registros_vendedores_abs.copy()
registros_vendedores_rel[['Ventas', 'Comision']] = registros_vendedores_rel[['Ventas', 'Comision']].replace({0:np.NaN})

16. Cálculo de cambios porcentuales. 

    **NOTA:** La función pct_change devuelve en el rango normal, no en el porcentual

In [ ]:
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 4)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 4)

17. Pivotear los datos.

In [ ]:
pivotear = registros_vendedores_rel.copy()

# Extraer el mes y el año de la columna "Fecha"
pivotear["Fecha"] = pd.to_datetime(pivotear["Fecha"])
pivotear["Month"] = pivotear["Fecha"].dt.month
pivotear["Year"] = pivotear["Fecha"].dt.year

# Convertir "Month" a string con formato de dos cifras
pivotear["Month"] = pivotear["Month"].apply(lambda x: str(x).zfill(2))

# Convertir "Year" a string y quedarse con los últimos 2 dígitos
pivotear["Year"] = pivotear["Year"].apply(lambda x: str(x)[-2:])

# Crear la variable "Fecha2" que concatena "Year" y "Month"
pivotear["Fecha2"] = pivotear["Year"] + pivotear["Month"]

# Eliminar columnas
pivotear.drop(columns=['Fecha','Ventas','Comision','Dato_original','Month','Year'], inplace=True)

# Renombrar la columnas
pivotear.rename(columns={'Fecha2': 'Fecha', 'Y_pct_Ventas': 'Y_pct_Ven', 'Y_pct_Comision': 'Y_pct_Com', 'F_pct_Ventas': 'F_pct_Ven', 'F_pct_Comision': 'F_pct_Com'}, inplace=True)

In [ ]:
# Realizar el pivoteo y la agrupación
pivot_df = pivotear.pivot_table(index=["ID",'Subrubro','Omega','Modelo'], columns=['Fecha'], values=['Y_pct_Ven', 'Y_pct_Com', 'F_pct_Ven', 'F_pct_Com'])

# Generar los nombres de las columnas finales
columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

# Asignar los nuevos nombres de columnas
pivot_df.columns = columns

# Restablecer el índice para que "ID" vuelva a ser una columna
pivot_df = pivot_df.reset_index()

18. Guaradamos todo lo hecho.

In [ ]:
pivot_df.to_csv('../data/interim/tp3_vendedores_vector.csv', index=False)